In [ ]:
from bs4 import BeautifulSoup
import requests
import json
from requests import Session
import re

def get_house():
    pass

def get_house_info(url):
    s = requests.Session()
    house_info = s.get(url)
    soup = BeautifulSoup(house_info.text, "html.parser")
 

    house_info={}
    # captures {<any text>} within window.classified
    regex = r"window.classified = (\{.*\})"
    
    # returns a list containing {<any text}
    script_text=soup.find('div',attrs={"id":"main-container"}).script.text
    # applies regex to scrip_text
    script_after_regex = re.findall(regex, script_text)
    
    # returns first element of script_after_regex (full clustered dictionary of property attributes)
    script_text_dict = json.loads(script_after_regex[0])
    properties = script_text_dict['property']

    # adds 'key : value' pairs to house_info dictionary. If value does not exist, it is replaced by a 0.
    try:
      house_info['id'] = script_text_dict['id']
    except:
      house_info['id'] = 0

    try:
      house_info['type'] = properties['type']
    except:
      house_info['type'] = 0

    try:
      house_info['subtype'] = properties['subtype']
    except:
      house_info['subtype'] = 0

    try:
      house_info['locality'] = properties['location']['locality']
    except:
      house_info['locality'] = 0

    try:
      house_info['price'] = script_text_dict['price']['mainValue']
    except:
      house_info['price'] = 0
    
    try:
      house_info['sale type'] = script_text_dict['price']['type']
    except:
      house_info['sale type'] = 0

    try:  
      house_info['number of bedrooms'] = properties['bedroomCount']
    except:
      house_info['number of bedrooms'] = 0

    try:  
      house_info['living room area'] = properties['livingRoom']['surface']
    except:
      house_info['living room area'] = 0

    try:
      house_info['fully equipped kitchen'] = script_text_dict['property']['kitchen']['type'].lower()
    except:
      house_info['fully equipped kitchen'] = 0

    try:
      house_info['furnished'] = script_text_dict['transaction']['sale']['isFurnished']
    except:
      house_info['furnished'] = 0

    try:
      house_info['fireplace'] = properties['fireplaceExists']
    except:
      house_info['fireplace'] = 0

    try:
      house_info['terrace'] = properties['hasTerrace']
    except:
      house_info['terrace'] = 0

    try:
      house_info['terrace area'] = properties['terraceSurface']
    except:
      house_info['terrace area'] = 0
    
    try:
      house_info['garden'] = properties['hasGarden']
    except:
      house_info['garden'] = 0

    try:
      house_info['garden area'] = properties['gardenSurface']
    except:
      house_info['garden area'] = 0

    try:
      house_info['total property area'] = properties['netHabitableSurface']
    except:
      house_info['total property area'] = 0

    try:
      house_info['total land area'] = properties['land']['surface']
    except:
      house_info['total land area'] = 0

    try:
      house_info['facades'] = properties['building']['facadeCount']
    except:
      house_info['facades'] = 0

    try:
      house_info['swimming pool'] = properties['hasSwimmingPool']
    except:
      house_info['swimming pool'] = 0

    try:
      house_info['doesnt need renovating'] = properties['building']['condition'].lower()
    except: 
      house_info['doesnt need renovating'] = 0

  
    if house_info['doesnt need renovating'] == "to_be_done_up" or house_info['doesnt need renovating'] == "to_renovate" or ['doesnt need renovating'] == "to_refurbish":
      house_info['doesnt need renovating'] = 0
    else:
      house_info['doesnt need renovating'] = 1
    if house_info['fully equipped kitchen'] == "undefined" or house_info['fully equipped kitchen'] == "uninstalled" or house_info['fully equipped kitchen'] == "semi_equipped" or house_info['fully equipped kitchen'] == "usa_uninstalled" or house_info['fully equipped kitchen'] == "usa_semi_equipped":
      house_info['fully equipped kitchen'] = 0
    else:
      house_info['fully equipped kitchen'] = 1
    for key in house_info.keys():
      if house_info[key] == None:
        house_info[key] = 0
      elif house_info[key] == False:
        house_info[key] = 0
      elif house_info[key] == True:
        house_info[key] = 1
      else:
        pass

    print(house_info)




In [ ]:
    """key_list = ['type', 'subtype' , 'locality' , 'price' , 'sale type' , 'number of bedrooms' , 'living room area' , 'kitchen status' , 'furnished' , 'fireplace' , 'terrace' , 
            'terrace area' , 'garden' , 'garden area' , 'total land area' , 'facades' , 'swimming pool' , 'current state']
    value_list = [script_text_dict['id'] , properties['type'] , properties['subtype'] , properties['location']['locality'] , script_text_dict['price']['mainValue'] ,
                  script_text_dict['price']['type'] , properties['bedroomCount'] ,  properties['livingRoom']['surface'] , properties['livingRoom']['surface'] , 
                  script_text_dict['property']['kitchen']['type'] , script_text_dict['transaction']['sale']['isFurnished'] , properties['fireplaceExists'] , 
                  properties['fireplaceExists'] , properties['hasTerrace'] , properties['terraceSurface'] , properties['hasGarden'] , properties['gardenSurface'] ,
                  properties['netHabitableSurface'] , properties['land']['surface'] , properties['building']['facadeCount'] , properties['hasSwimmingPool'] , 
                  properties['building']['condition']]
    print(key_list)
    print(value_list)
    for i in range(len(key_list)):
        try:
          house_info[key_list[i]] = value_list[i]
        except:
            house_info[key_list[i]] = 0"""

#### Attributes to include in a single property's data.
* Locality
* Type of property (House/apartment)
* Subtype of property (Bungalow, Chalet, Mansion, ...)
* Price
* Type of sale (Exclusion of life sales)
* Number of bedrooms
* Living Area
* Fully equipped kitchen (Yes/No)
* Furnished (Yes/No)
* Open fire (Yes/No)
* Terrace (Yes/No)
* If yes: Area
* Garden (Yes/No)
* If yes: Area
* Surface of the land
* Surface area of the plot of land 
* Number of facades
* Swimming pool (Yes/No)
* State of the building (New, to be renovated, ...)

In [ ]:
# testing
gent_appartment_url = "https://www.immoweb.be/en/classified/penthouse/for-sale/gent/9000/10611711"
brussels_house_url = "https://www.immoweb.be/en/classified/villa/for-rent/brussels/1020/10659745"
brugge_loft = "https://www.immoweb.be/en/classified/loft/for-sale/brugge/8000/10630907"

get_house_info(gent_appartment_url)
get_house_info(brussels_house_url)
get_house_info(brugge_loft)

In [ ]:
def get_url(i=1):
    root_house_list_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&isNewlyBuilt=false&isALifeAnnuitySale=false&isAPublicSale=false&page={i}&orderBy=relevance"
    s = requests.Session()
    house_list = s.get(root_house_list_url)
    soup = BeautifulSoup(house_list.text, "html.parser")
    url_list = []

    for block in soup.find_all("h2",{'class':"card__title card--result__title"}):
        tag_a = block.find('a')['href']
        url_list.append(tag_a)
    
    return(url_list)



In [ ]:
def get_house_data()

In [ ]:
s = requests.Session()
house_info = s.get("https://www.immoweb.be/en/classified/penthouse/for-sale/gent/9000/10611711")
print(type(house_info.text))
print(house_info.content)


In [ ]:
list_of_url_lists=[]
for x in range(1,201):
    list_of_url_lists.append(get_url(i=x))
    

In [ ]:
print(list_of_url_lists)

In [ ]:
print(list_of_url_lists[0])